In [1]:
import torch
from ettcl.core.pipelines import ClassificationPipeline
from datasets import load_from_disk, Split
from sklearn.metrics import accuracy_score

text_column = 'text'
label_column = 'label'

In [2]:
dataset = load_from_disk("/home/IAIS/hiser/data/trec-6")
dataset.set_format('torch')

train_dataset = dataset[Split.TRAIN]
test_dataset = dataset[Split.TEST].select(range(500))

label_names = train_dataset.features[label_column].names

In [10]:
pipeline = ClassificationPipeline(
    "/home/IAIS/hiser/dev/hyped/examples/output/trec_6_bert/model/best-model",
    architecture='colbert',
    # tokenizer_kwargs={'query_maxlen': 16, 'doc_maxlen': 16},
    tokenizer_kwargs={'query_token': None, 'doc_token': None, 'add_special_tokens': False},
    # indexer_kwargs={'num_partitions': 2048},
    searcher_kwargs={'ncells': 16},
    # searcher_kwargs={'ncells': 1},
    k=5,
    # num_sentences=32,
)
pipeline.train_index(train_dataset)
# pipeline.train_index(train_dataset, index_path="outputs/index_scolbert")

Some weights of the model checkpoint at /home/IAIS/hiser/dev/hyped/examples/output/trec_6_bert/model/best-model were not used when initializing ColBERTModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing ColBERTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ColBERTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
#> Will delete 14 files already at outputs/index_colbert
[0][2023-07-27 16:25:16,609] [INFO] [init] nranks = 2 	 gpus = [0, 1] 	 device=0
[1][2023-07-27 16:25:20,669] [INFO] [init] nranks = 2 	 gpus = [0, 1] 	 device=1
Encoding: 100%|██████████| 10/10 [00:01<00:00,  8.33it/s]
[0][2023-07-27 16:25:23,484] [INFO] [print_message] [0] 		 avg_do

Clustering 53432 points in 768D to 2048 clusters, redo 1 times, 4 iterations
  Preprocessing in 0.04 s
  Iteration 3 (0.17 s, search 0.14 s): objective=5905.76 imbalance=1.567 nsplit=0       


WARNING clustering 53432 points to 2048 centroids: please provide at least 79872 training points
[0][2023-07-27 16:25:26,611] [INFO] [print_message] Loading decompress_residuals_cuda_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
[0][2023-07-27 16:25:26,733] [INFO] [print_message] Loading packbits_cuda_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
[0][2023-07-27 16:25:26,860] [INFO] [print_message] #> Got bucket_cutoffs_quantiles = tensor([0.2500, 0.5000, 0.7500], device='cuda:0') and bucket_weights_quantiles = tensor([0.1250, 0.3750, 0.6250, 0.8750], device='cuda:0')
[0][2023-07-27 16:25:26,861] [INFO] [print_message] #> Got bucket_cutoffs = tensor([-6.7596e-03,  3.0518e-05,  6.8665e-03], device='cuda:0') and bucket_weights = tensor([-0.0129, -0.0030,  0.0031,  0.0131], device='cuda:0')
[0][2023-07-27 16:25:26,861] [INFO] [print_message] avg_residual = 0.00933074951171875
[1][2023-07-27 16:25:26,900] [INFO] [print_messag

In [20]:
pipeline.k = 1
preds = pipeline.predict(test_dataset[text_column], test_dataset[label_column])
accuracy_score(test_dataset[label_column], preds)

Setting TOKENIZERS_PARALLELISM=false for forked processes.


Searching (num_proc=2):   0%|          | 0/500 [00:00<?, ? examples/s]

0.916

In [21]:
torch.where(preds != test_dataset[label_column])[0]

tensor([  4,  34,  38,  56,  90, 117, 135, 138, 162, 176, 190, 193, 223, 241,
        251, 275, 280, 291, 296, 298, 323, 325, 332, 356, 358, 365, 376, 384,
        398, 407, 408, 414, 425, 437, 439, 441, 454, 455, 457, 460, 483, 494])

In [30]:
maxsim_view = pipeline.explain_prediction(0, 0, max_only=True)
maxsim_view

Query 0: How far is it from Denver to Aspen ?
True Label: NUM, predicted label: NUM
Match 0: How many miles is it from London , England to Plymouth , England ?


In [8]:
with open("outputs/maxsim_view.html", 'w') as file:
    file.write(maxsim_view.data)